<h4 align=center>Problem Statement</h4>

<p>Segment the customers based on their transactional behaviour. Use the Online retail dataset available freely at: http://archive.ics.uci.edu/ml/datasets/online+retail</p>

<h6>Additional Information</h6>

<p>This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.</p>

<h6>Data Dictionary</h6>

<p>InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
Description: Product (item) name. Nominal. 
Quantity: The quantities of each product (item) per transaction. Numeric.	
InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated. 
UnitPrice: Unit price. Numeric, Product price per unit in sterling. 
CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. 
Country: Country name. Nominal, the name of the country where each customer resides.</p>

<h6>Importing packages and visualizing the dataset</h6>

In [30]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn import *
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel('./Datasets/Online_Retail.xlsx')

In [3]:
## Checking the dataset top 5 rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
## Checking the count of each column in dataset
df.count()

InvoiceNo      541909
StockCode      541909
Description    540455
Quantity       541909
InvoiceDate    541909
UnitPrice      541909
CustomerID     406829
Country        541909
dtype: int64

In [5]:
#Checking for null values count
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df_copy=df

In [7]:
#Creating redable date column from Invoice date
df['date']=df['InvoiceDate'].dt.date

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


In [9]:
df.date.min(), df.date.max()

(datetime.date(2010, 12, 1), datetime.date(2011, 12, 9))

In [10]:
### Data is availabe from 1st Dec 2010 to 9 dec 2011

In [11]:
###Quantity and Unit Price can not be neagtive, removing abnormal values also we don't need nul cust IDs
df_filtered=df[(df.Quantity>0) & (df.UnitPrice>0.0)]

In [12]:
df_filtered=df_filtered[pd.notnull(df_filtered['CustomerID'])]

In [13]:
df_filtered.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
date           0
dtype: int64

In [14]:
### Customer distribution across countries
customer_country=df_filtered[['Country','CustomerID']].drop_duplicates()

In [15]:
customer_country.count()

Country       4346
CustomerID    4346
dtype: int64

In [16]:
customer_country.groupby(['Country']).count().reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
35,United Kingdom,3920
14,Germany,94
13,France,87
30,Spain,30
3,Belgium,25
32,Switzerland,21
26,Portugal,19
18,Italy,14
12,Finland,12
1,Austria,11


In [17]:
### Maximum Customers are from UK only, filtering it for UK base only
df_filtered=df_filtered[(df_filtered.Country=='United Kingdom')]

In [18]:
df_filtered.count()

InvoiceNo      354321
StockCode      354321
Description    354321
Quantity       354321
InvoiceDate    354321
UnitPrice      354321
CustomerID     354321
Country        354321
date           354321
dtype: int64

In [19]:
df_filtered.shape
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354321 entries, 0 to 541893
Data columns (total 9 columns):
InvoiceNo      354321 non-null object
StockCode      354321 non-null object
Description    354321 non-null object
Quantity       354321 non-null int64
InvoiceDate    354321 non-null datetime64[ns]
UnitPrice      354321 non-null float64
CustomerID     354321 non-null float64
Country        354321 non-null object
date           354321 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.0+ MB


In [20]:
##Defining a function for count of unique values in each columns
def unique_num(df):
    for i in df.columns:
        count=df[i].nunique()
        print(i,":",count)

In [21]:
unique_num(df_filtered)

InvoiceNo : 16646
StockCode : 3645
Description : 3844
Quantity : 293
InvoiceDate : 15612
UnitPrice : 402
CustomerID : 3920
Country : 1
date : 305


In [22]:
### Calculation of additional columns required for RFM

In [23]:
df_filtered['amount']=df_filtered['Quantity']*df_filtered['UnitPrice']

In [24]:
df_filtered.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [25]:
df_filtered[(df.CustomerID==12346)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,amount
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,2011-01-18,77183.6


In [26]:
##Recency=Min number of days since last purchase, Data availble till 2011/12/9, so calculation date will be 2011-12-10
##Frequncy: nuber of orders placed
##Monetory: Amount spent

In [27]:
df_filtered['End_date']='2011-12-10'

In [28]:
df_filtered.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,amount,End_date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30,2011-12-10
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00,2011-12-10
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10


In [48]:
from datetime import datetime
import numpy as np

In [33]:
df_filtered['End_date']=df_filtered['End_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [42]:
df_filtered['date']=df_filtered['date'].apply(lambda x: str(x))

In [43]:
df_filtered['date']=df_filtered['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [46]:
df_filtered['Recency']= (df_filtered['End_date'] - df_filtered['date'])
#date(df_filtered['date'])-date(df_filtered['End_date'])

In [49]:
df_filtered['Recency']=df_filtered['Recency'].apply(lambda x: x / np.timedelta64(1, 'D'))

In [50]:
df_filtered.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,amount,End_date,Recency
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30,2011-12-10,374.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10,374.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00,2011-12-10,374.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10,374.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34,2011-12-10,374.0


In [55]:
rfm_Cal=df_filtered.groupby('CustomerID', as_index=False).agg({'Recency': lambda x: x.min(),
                                                                   'InvoiceNo': lambda x: len(x), 
                                                                   'amount': lambda x: x.sum()})

In [59]:
rfm_Cal=rfm_Cal.rename(columns={"Recency": "Recency", "InvoiceNo": "Frequency", "amount": "Monetory"})

In [60]:
rfm_Cal.head()

,CustomerID,Recency,Frequency,Monetory
0,12346.0,326.0,1,77183.60
1,12747.0,3.0,103,4196.01
2,12748.0,1.0,4595,33719.73
3,12749.0,4.0,199,4090.88
4,12820.0,4.0,59,942.34
